In [2]:
import json

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)

In [4]:
# documents = []

# for course_dict in docs_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
from openai import OpenAI

In [9]:
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'To execute a command in a running Docker container, you can use the `docker exec` command. Below are the general steps and examples for doing this:\n\n1. **Identify the Running Container**:\n   First, you need to identify the container ID or name of the running container in which you want to execute a command. You can list all running containers using the `docker ps` command:\n   ```sh\n   docker ps\n   ```\n\n2. **Execute a Command Inside the Container**:\n   Use the `docker exec` command along with the container ID or name to run a command inside the container. The basic syntax is as follows:\n   ```sh\n   docker exec [OPTIONS] CONTAINER COMMAND [ARG...]\n   ```\n\n   For example, to execute a simple command like `ls` in a container named `my_container`, you would run:\n   ```sh\n   docker exec my_container ls\n   ```\n\n   If you need an interactive shell session (e.g., Bash) inside the container, you can use the `-it` flags:\n   ```sh\n   docker exec -it my_container /bin/bash\n  

In [13]:
q="How do I execute a command in a running docker container?"

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
rag(query)

"To run Kafka, follow the relevant instructions based on your use case:\n\n### For Java:\nNavigate to your project directory and use the following command in the terminal to run a Kafka producer/consumer/KStreams, etc.:\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\nReplace `<jar_name>` with the actual name of your JAR file.\n\n### For Python:\nIf you're running Python Kafka, ensure you have your virtual environment set up and activate it. Here’s how you can do it:\n\n1. **Create a virtual environment and install required packages (run only once):**\n    ```shell\n    python -m venv env\n    source env/bin/activate\n    pip install -r ../requirements.txt\n    ```\n\n2. **Activate the virtual environment (run this every time you need it):**\n    ```shell\n    source env/bin/activate\n    ```\n\n3. **Deactivate the virtual environment when done:**\n    ```shell\n    deactivate\n    ```\n\nNote: On Windows, the activation c

In [17]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit homework assignments, but please be mindful of the deadlines for the final projects to ensure you complete everything on time.'

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
from elasticsearch import Elasticsearch
import os

In [18]:
es_user = os.getenv('ELASTIC_SEARCH_USER')
es_password = os.getenv('ELASTIC_SEARCH_PASSWORD')

# Create the Elasticsearch client with HTTPS
es_client = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=(es_user, es_password),
    verify_certs=False  # Use False only for self-signed certificates, set to True in production with valid certs
)

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/fy/b42gn4m17s5f5_1pbnt4kg3m0000gn/T/ipykernel_64307/3247640667.py:5: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch(


In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
from tqdm.auto import tqdm

In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/

In [26]:
query = 'How do I execute a command in a running docker container?'

In [29]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    return response

In [43]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    # Extracting the top score
    # if response['hits']['hits']:
    #     top_score = response['hits']['hits'][0]['_score']
    # else:
    #     top_score = None
    
    # return top_score
    return response


In [44]:
result_docs= elastic_search(query)
result_docs

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 690, 'relation': 'eq'}, 'max_score': 84.17781, 'hits': [{'_index': 'course-questions', '_id': 'pmfPRJABHgl564Smvqj7', '_score': 84.17781, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'WmfsRJABHgl564SmLKz-', '_score': 84.17781, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    # Extracting the top score
    if response['hits']['hits']:
        top_score = response['hits']['hits'][0]['_score']
    else:
        top_score = None
    
    return top_score

In [34]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag(query)

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'To execute a command in a running Docker container, you can utilize the following example provided for running `pgcli` inside a Docker container:\n\n```bash\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nIn this command:\n\n- `docker run -it --rm` starts a new interactive terminal session and removes the container once the command finishes.\n- `--network pg-network` specifies the Docker network to which the container should connect.\n- `ai2ys/dockerized-pgcli:4.0.1` is the Docker image being used.\n- `175dd47cda07:/#` indicates the command prompt inside the running container.\n- `pgcli -h pg-database -U root -p 5432 -d ny_taxi` is the command being executed inside the container, connecting to the PostgreSQL database.\n\nReplace the values with those relevant to your use case if needed.'

In [30]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [73]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [74]:
def build_prompt(query, search_results):
    prompt_template = """
                        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
                        Use only the facts from the CONTEXT when answering the QUESTION.
                        
                        QUESTION: {question}
                        
                        CONTEXT: 
                        {context}
                    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [80]:
def build_prompt(query, search_results):
    # Define the context template
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    # Build the context from search results
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    # Define the prompt template
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    # Build the final prompt
    prompt = prompt_template.format(question=query, context=context.strip())
    return prompt

In [81]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(len(prompt))
    encoded_tokens = encoding.encode(prompt)

# Get the number of tokens
    num_tokens = len(encoded_tokens)
    print(num_tokens)
    answer = llm(prompt)
    return answer

In [82]:
rag(query)

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1744
375


'To copy files from a different folder into a Docker container’s working directory, you can use the Docker `COPY` command in your Dockerfile. This command allows you to specify the files you want to copy and the destination directory in the container. Here’s the basic syntax:\n\n```\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\n```\n\nAdditionally, you can use the `docker cp` command to copy files from your local machine into a running Docker container. The basic syntax for this is:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThese methods will enable you to copy the required files into your Docker container’s working directory.'

In [53]:
query ="How do I copy files from a different folder into docker container’s working directory?"

In [59]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.7/906.7 kB 12.0 MB/s eta 0:00:00m eta 0:00:0136m0:00:01


In [60]:
encoding.decode_single_token_bytes(63842)

NameError: name 'encoding' is not defined

In [63]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [62]:
import tiktoken

In [64]:
encoding

<Encoding 'o200k_base'>

In [ ]:
encoded_tokens = encoding.encode(prompt)

# Get the number of tokens
num_tokens = len(encoded_tokens)